In [1]:
from modules.smpl_model._smpl import SMPL as SMPL_metro, H36M_J17_NAME, H36M_J17_TO_J14
from modules.utils.image_utils import plot_tensor, to_tensor, visualize_mesh
from modules.smpl_model.config_smpl import *
from modules.smpl_model.smpl_pose2mesh import SMPL
from modules.datasets.dataset_3DPW import get_data as get_data_3dpw
from modules.datasets.dataset_H36M import get_data as get_data_h36m
from modules.datasets.FullDataset import get_full_train_val_data
from modules.utils.data_utils_3dpw import get_chunks_img_paths_list_seq
from modules.utils.data_utils_h36m import get_data_list_h36m, get_data_chunk_list_h36m
from modules.smpl_model.smpl_pose2mesh import SMPL
from modules.utils.geometry import world2cam

In [2]:
import torch
import os.path as osp
import json
import copy
import numpy as np
import os
import pickle as pkl
from PIL import Image
import torch
import random

## SMPL Model Pose2Mesh

In [3]:
smpl = SMPL()
smpl_metro = SMPL_metro()

# Adapt Regressor for 10 PCs
smpl.layer['neutral'].th_shapedirs= smpl.layer['neutral'].th_shapedirs[:,:,:10]
smpl.layer['neutral'].th_betas= smpl.layer['neutral'].th_betas[:,:10]

In [4]:
smpl.joint_regressor.shape

(29, 6890)

In [5]:
smpl_metro.weights.shape

torch.Size([6890, 24])

In [ ]:
datalist1 = get_data_list_h36m('../H36M/annotations', [1], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

In [ ]:
datalist5 = get_data_list_h36m('../H36M/annotations', [5], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 396316/396316 [00:23<00:00, 16594.12it/s]


In [ ]:
datalist6 = get_data_list_h36m('../H36M/annotations', [6], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 249864/249864 [00:22<00:00, 11119.02it/s]


In [ ]:
datalist7 = get_data_list_h36m('../H36M/annotations', [7], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 406484/406484 [00:38<00:00, 10522.99it/s]


In [ ]:
datalist8 = get_data_list_h36m('../H36M/annotations', [8], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 258712/258712 [00:28<00:00, 8984.27it/s] 


In [ ]:
datalist9 = get_data_list_h36m('../H36M/annotations', [9], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 317668/317668 [00:45<00:00, 7000.11it/s] 


In [3]:
datalist11 = get_data_list_h36m('../H36M/annotations', [11], 25, store_as_pkl=False,out_dir='../H36M/data_pickle_kp3d')

100%|██████████| 231151/231151 [00:06<00:00, 34821.27it/s]


In [ ]:
data_list_complete = []
for datalist in [datalist1, datalist5, datalist6, datalist7, datalist8]:
    for data in datalist:
        data_list_complete.append(data)
     


In [ ]:
data_list_complete2 = []
for datalist in [datalist9, datalist11]:
    for data in datalist:
        data_list_complete2.append(data)
     

In [ ]:
len(data_list_complete), len(data_list_complete2)

(309300, 74032)

In [4]:
out_dir='../H36M/data_pickle_kp3d'
fitting_thr = 25

In [ ]:
sub_str = f'9to11'
with open(osp.join(out_dir, f'datalist_h36m_thr{fitting_thr}_{sub_str}subj.pickle'), 'wb') as fp:
     pkl.dump(data_list_complete2, fp)

In [ ]:
a, b = get_data_chunk_list_h36m(annot_dir=None,
                    subject_list=[1,5,6,7,8],
                    fitting_thr=25,
                    len_chunks=8,
                    load_seq_datalist=None,
                    load_datalist='/Users/clara/Desktop/MeineProjekte/H36M/data_pickle_kp3d/datalist_h36m_thr25_1to8subj.pickle',
                    store_as_pkl=False,
                    out_dir='/Users/clara/Desktop/MeineProjekte/H36M/data_pickle_kp3d/',
                    seq_datalist=None,
                    )

load datalist


In [ ]:
out_dir = '/Users/clara/Desktop/MeineProjekte/H36M/data_pickle_kp3d/'
fitting_thr=25
sub_str = '1to8'

In [ ]:
with open(osp.join(out_dir, f'seq_datalist_h36m_thr{fitting_thr}_{sub_str}subj.pickle'), 'wb') as fp:
    pkl.dump(b, fp)

## Dataset H36M

In [4]:
datah36m = get_data_h36m(data_path='../H36M',
                        subject_list = [11],
                        img_size=224,
                        load_datalist='../H36M/data_pickle_kp3d/datalist_h36m_thr25_11to11subj.pickle', 
                        load_from_zarr=['../H36M/img_zarr/imgs_h36m_resnet_thr25_11to11subj.zarr'],
                        mask=True,
                        smpl=smpl.layer['neutral'],
                        backgrounds = None,
                        store_images=False)

datasets initialized
load datalist
datalist h36m, current memory 0.9011497497558594 GB
11 ../H36M/img_zarr/imgs_h36m_resnet_thr25_11to11subj.zarr
data h36m, current memory 0.901153564453125 GB
cpu


In [5]:
loader_trn = torch.utils.data.DataLoader(
        dataset=datah36m,
        batch_size=3,
        shuffle=True,
    )

In [6]:
dat = next(iter(loader_trn))

In [7]:
img = dat['img']
img_path= dat['img_path']
verts = dat['vertices']
cam_pose =dat['cam_pose']
cam_intr = dat['cam_intr']
trans = dat['trans']
joints = np.array(dat['joints_3d'])

In [8]:
joints_p2m = smpl.get_joints(verts)

In [12]:
joints_p2m[:,:24,:].shape

torch.Size([3, 24, 3])

In [ ]:
joints_metro = smpl_metro.get_h36m_joints(verts)
joints_metro = joints_metro - joints_metro[:, H36M_J17_NAME.index('Pelvis'),:][:, None, :]
joints_metro = np.array(joints_metro[:, H36M_J17_TO_J14, :])

In [ ]:
orig_img = np.array(Image.open(img_path))
orig_img = to_tensor(orig_img)

In [ ]:
rend_img = visualize_mesh(orig_img, smpl=smpl_metro, cam_intr=cam_intr, vertices=(verts+trans)[None])
plot_tensor(rend_img)

In [ ]:
plot_tensor(img)

## Full Dataset

In [35]:
from modules.datasets.FullDataset import get_full_train_val_data
from modules.datasets.dataset_3DPW import get_data

In [36]:
data3dpw = get_data('../3DPW', 
'validation', 8, 
True,
 False, 
'/Users/clara/Desktop/MeineProjekte/3DPW/img_zarr/imgs_3dpw_valid.zarr',
224,
'/Users/clara/Desktop/MeineProjekte/3DPW/data_pickle/ids_imgpaths_seq/ids_paths_seq_validation_min8_kps.pickle',smpl=smpl.layer['neutral'])

In [37]:
loader_trn = torch.utils.data.DataLoader(dataset=data3dpw,
                                            batch_size=16,
                                            shuffle=True,)

In [58]:
data = next(iter(loader_trn))
joints = np.array(data['joints_3d'])
vertices = data['vertices']

In [59]:
joints_metro = smpl_metro.get_h36m_joints(vertices)
joints_metro = joints_metro - joints_metro[:, H36M_J17_NAME.index('Pelvis'),:][:, None, :]
joints_metro = np.array(joints_metro[:, H36M_J17_TO_J14, :])

In [61]:
np.mean(joints[0],0)[None,:]

array([[-1.2062482 , -0.12041646,  3.7628517 ]], dtype=float32)

In [79]:
def get_fitting_error(joint3d_h36m_gt, joint3d_smpl):
    #joint3d_h36m_gt = joint3d_h36m_gt - joint3d_h36m_gt[H36M_J17_NAME.index('Pelvis'), None,:] # root-relative
    # translation alignment
    #joint3d_smpl = joint3d_smpl - np.mean(joint3d_smpl,0)[None,:] + np.mean(joint3d_h36m_gt,0)[None,:]
    error = np.sqrt(np.sum((joint3d_h36m_gt - joint3d_smpl)**2, 1)).mean()
    return error

In [63]:
get_fitting_error(joints[0],joints_metro[0])

[[ 0.10538352  0.86494523 -0.1566712 ]
 [ 0.06657241  0.44574255  0.01868278]
 [ 0.11751296 -0.0185063   0.09091746]
 [-0.11842925  0.01930699 -0.08864043]
 [-0.12712356  0.4465999   0.11958259]
 [-0.01726985  0.8680906   0.21322313]
 [ 0.20283544 -0.00961037  0.32306966]
 [ 0.17730615 -0.2043225   0.16353977]
 [-0.00404913 -0.42414302  0.14264494]
 [-0.21976408 -0.37624747 -0.01301892]
 [-0.21796632 -0.11224277 -0.13877317]
 [-0.19333783  0.14599812 -0.19696826]
 [-0.13562497 -0.44751734  0.08347292]
 [-0.22294733 -0.5827157   0.15034515]]
[[-1.058943    0.7005732   3.5553658 ]
 [-1.0977542   0.28137052  3.7307198 ]
 [-1.0468136  -0.18287832  3.8029544 ]
 [-1.2827559  -0.14506502  3.6233966 ]
 [-1.2914501   0.28222787  3.8316195 ]
 [-1.1815964   0.7037186   3.92526   ]
 [-0.96149117 -0.17398238  4.0351067 ]
 [-0.9870205  -0.3686945   3.8755767 ]
 [-1.1683757  -0.58851504  3.854682  ]
 [-1.3840907  -0.5406195   3.699018  ]
 [-1.382293   -0.2766148   3.573264  ]
 [-1.3576645  -0.0183738

0.55169505

### Chunks 

In [13]:
import pickle as pkl

from modules.utils.image_utils import to_tensor, transform, transform_visualize, crop_box

from modules.datasets.dataset_seq_3DPW import SequenceWise3DPW
from modules.datasets.dataset_seq_H36M import SequenceWiseH36M
from modules.datasets.FullDataset import get_full_seq_train_val_data

In [58]:
data_h36m = SequenceWiseH36M(
            data_path='../H36M',
            subject_list=[9,11],
            smpl=smpl.layer['neutral'],
            load_from_zarr=None,
            load_seq_datalist='/Users/clara/Desktop/MeineProjekte/H36M/data_pickle/seq_datalist_h36m_thr25_9to11subj.pickle',
            load_datalist=None,
            len_chunks=8,
            img_size=224,
            mask= False,
            fitting_thr=25,
            store_images=False,
            backgrounds=None,
            )

In [59]:
data = data_h36m[1]

In [68]:
data_3dpw = SequenceWise3DPW(data_path='../3DPW',
        split='train',
        num_required_keypoints=8,
        smpl=smpl.layer['neutral'],
        len_chunks=8,
        store_sequences=True,
        store_images=False,
        load_from_zarr='/Users/clara/Desktop/MeineProjekte/3DPW/img_zarr/imgs_3dpw_train.zarr',
        img_size=224,
        load_chunks_seq='/Users/clara/Desktop/MeineProjekte/3DPW/data_pickle/img_seqs_list_paths_seq/img_seqs_list_paths_seq_train_min8_kps.pickle',
    )

In [ ]:
data = data_3dpw[0]

In [14]:
full_data = get_full_seq_train_val_data(
        dataset='full',
        data_path_3dpw='../3DPW',
        len_chunks=8,
        num_required_keypoints = 8,
        store_sequences=True,
        store_images_3dpw=True,
        load_from_zarr_3dpw_trn=None,
        load_from_zarr_3dpw_val=None,
        img_size=224,
        load_chunks_seq_trn='/Users/clara/Desktop/MeineProjekte/3DPW/data_pickle/img_seqs_list_paths_seq/img_seqs_list_paths_seq_train_min8_kps.pickle',
        load_chunks_seq_val='/Users/clara/Desktop/MeineProjekte/3DPW/data_pickle/img_seqs_list_paths_seq/img_seqs_list_paths_seq_validation_min8_kps.pickle',
        data_path_h36m='../H36M',
        store_images_h36m=False,
        load_from_zarr_h36m_trn=None,
        load_from_zarr_h36m_val=None,
        load_seq_datalist_trn='/Users/clara/Desktop/MeineProjekte/H36M/data_pickle/seq_datalist_h36m_thr25_9to11subj.pickle',
        load_seq_datalist_val='/Users/clara/Desktop/MeineProjekte/H36M/data_pickle/seq_datalist_h36m_thr25_9to11subj.pickle',
        backgrounds=None,
        mask=True,
        val_on_h36m=True,
        subject_list_trn=[9,11],
        subject_list_val=[9,11],
        fitting_thr=25,
    )

initialize smpl model
get 3dpw train data
get 3dpw validation data
get h36m train data
get h36m validation data
length train data: 3dpw: 2828, h36m: 9148, total: 11976
length validation data: 3dpw: 1294, h36m: 9148, total: 10442


In [15]:
train_data = full_data[0]
val_data= full_data[1]

In [8]:
data_loader = torch.utils.data.DataLoader(dataset=data.set_chunks(), 
                                        batch_size=batch_size_val, 
                                        shuffle=False,)

dict_keys(['img_paths', 'imgs', 'betas', 'poses', 'trans', 'vertices', 'cam_pose', 'cam_intr'])